In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
#binarize the labels
lb = LabelBinarizer()

import pandas as pd

data_files = ['cle_train.csv','cle_test.csv','hun_train.csv','hun_test.csv','swi_train.csv','swi_test.csv','vir_train.csv','vir_test.csv']

datasets = {}

for file in data_files:
    data = pd.read_csv('../TrainTestData/' + file)
    
    X = data.iloc[:, :-1]
    Y = data.iloc[:, -1]
    
    Y_binary = Y.apply(lambda x: 1 if x > 0 else 0)
    
    # Extract the name from the file path
    name = file.split('.')[0]
    
    # Store the dataset components in a dictionary
    datasets[name] = {'X': X, 'Y': Y, 'Y_binary': Y_binary}

# Unpack the dictionary values in a loop
variables = ['cle', 'hun', 'swi', 'vir']
train_test = ['train', 'test']

for var in variables:
    for tt in train_test:
        X, Y, Y_binary = datasets[f'{var}_{tt}'].values()
        globals()[f'{var}_X_{tt}'] = X
        globals()[f'{var}_Y_{tt}'] = Y
        globals()[f'{var}_Y_{tt}_binary'] = Y_binary

In [3]:
X_test = pd.concat([cle_X_test,hun_X_test,swi_X_test,vir_X_test])
y_test = pd.concat([cle_Y_test_binary,hun_Y_test_binary,swi_Y_test_binary,vir_Y_test_binary])

X_train = pd.concat([cle_X_train,hun_X_train,swi_X_train,vir_X_train])
y_train = pd.concat([cle_Y_train_binary,hun_Y_train_binary,swi_Y_train_binary,vir_Y_train_binary])

In [4]:
def create_clients():
    cle_zip = list(zip(cle_X_train.values,cle_Y_train_binary))
    hun_zip = list(zip(hun_X_train.values,hun_Y_train_binary))
    vir_zip = list(zip(vir_X_train.values,vir_Y_train_binary))
    swi_zip = list(zip(swi_X_train.values,swi_Y_train_binary))
    
    shards = [cle_zip, hun_zip, vir_zip,swi_zip]
    client_names = ["client_1","client_2","client_3","client_4"]
    dic = {client_names[i] : shards[i] for i in range(len(client_names))}
    return dic


def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)


class DNN:
    @staticmethod
    def build(shape, classes):

        model = Sequential()
        model.add(Dense(64, input_shape=(22,), activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final


def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    length = len(y_test)
    Y_test = tf.reshape(Y_test,(length,1))
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), Y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [5]:
#create clients
clients = create_clients()

#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 100
    
#create optimizer
lr = 0.01 
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.SGD(lr=lr, decay=lr / comms_round, momentum=0.9) 

#initialize global model
smlp_global = DNN()
global_model = smlp_global.build(22, 2)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = DNN()
        local_model = smlp_local.build(22, 2)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(250)
        smlp_SGD = DNN()
        SGD_model = smlp_SGD.build(22, 2) 

        SGD_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

D:\Anaconda\envs\GPU\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


9/9 [==============================] - 0s 750us/step
comm_round: 0 | global_acc: 70.758% | global_loss: 0.6854538917541504
9/9 [==============================] - 0s 625us/step
comm_round: 1 | global_acc: 79.061% | global_loss: 0.6804894804954529
9/9 [==============================] - 0s 750us/step
comm_round: 2 | global_acc: 79.061% | global_loss: 0.6756334900856018
9/9 [==============================] - 0s 752us/step
comm_round: 3 | global_acc: 79.422% | global_loss: 0.6705135703086853
9/9 [==============================] - 0s 625us/step
comm_round: 4 | global_acc: 81.227% | global_loss: 0.6654118299484253
9/9 [==============================] - 0s 750us/step
comm_round: 5 | global_acc: 83.032% | global_loss: 0.6599116921424866
9/9 [==============================] - 0s 625us/step
comm_round: 6 | global_acc: 83.032% | global_loss: 0.6538342237472534
9/9 [==============================] - 0s 875us/step
comm_round: 7 | global_acc: 82.310% | global_loss: 0.6479371190071106
9/9 [===========

9/9 [==============================] - 0s 625us/step
comm_round: 66 | global_acc: 86.643% | global_loss: 0.533951997756958
9/9 [==============================] - 0s 625us/step
comm_round: 67 | global_acc: 86.643% | global_loss: 0.5333611369132996
9/9 [==============================] - 0s 750us/step
comm_round: 68 | global_acc: 85.921% | global_loss: 0.5329806208610535
9/9 [==============================] - 0s 625us/step
comm_round: 69 | global_acc: 86.282% | global_loss: 0.53282231092453
9/9 [==============================] - 0s 625us/step
comm_round: 70 | global_acc: 85.560% | global_loss: 0.5322763323783875
9/9 [==============================] - 0s 625us/step
comm_round: 71 | global_acc: 85.921% | global_loss: 0.5319004654884338
9/9 [==============================] - 0s 625us/step
comm_round: 72 | global_acc: 85.560% | global_loss: 0.5312533378601074
9/9 [==============================] - 0s 750us/step
comm_round: 73 | global_acc: 85.921% | global_loss: 0.5310496091842651
9/9 [======

In [6]:
Y_predictions = np.argmax(SGD_model.predict(X_test),axis = 1)

9/9 [==============================] - 0s 625us/step


In [7]:
cm = confusion_matrix(Y_predictions, Y_test)
cm

array([[102,  12],
       [ 21, 142]], dtype=int64)

In [9]:
print(classification_report(Y_test, Y_predictions, digits=4))

              precision    recall  f1-score   support

           0     0.8947    0.8293    0.8608       123
           1     0.8712    0.9221    0.8959       154

    accuracy                         0.8809       277
   macro avg     0.8830    0.8757    0.8783       277
weighted avg     0.8816    0.8809    0.8803       277



# Testing on each dataset

In [10]:
Y_cle = np.argmax(SGD_model.predict(cle_X_test),axis = 1)
cm_cle = confusion_matrix(Y_cle, cle_Y_test_binary)
print(cm_cle)
print(classification_report(Y_cle, cle_Y_test_binary, digits=4))

3/3 [==============================] - 0s 1ms/step
[[42  4]
 [ 9 36]]
              precision    recall  f1-score   support

           0     0.8235    0.9130    0.8660        46
           1     0.9000    0.8000    0.8471        45

    accuracy                         0.8571        91
   macro avg     0.8618    0.8565    0.8565        91
weighted avg     0.8613    0.8571    0.8566        91



In [11]:
mismatch = [i for i, (a,b) in enumerate(zip(Y_cle, cle_Y_test_binary)) if a != b]
print(mismatch)

[19, 28, 30, 32, 38, 51, 53, 60, 64, 67, 69, 70, 77]


In [12]:
Y_vir = np.argmax(SGD_model.predict(vir_X_test),axis = 1)
cm_vir = confusion_matrix(Y_vir, vir_Y_test_binary)
print(cm_vir)
print(classification_report(Y_vir, vir_Y_test_binary, digits=4))

2/2 [==============================] - 0s 1ms/step
[[ 6  3]
 [ 7 44]]
              precision    recall  f1-score   support

           0     0.4615    0.6667    0.5455         9
           1     0.9362    0.8627    0.8980        51

    accuracy                         0.8333        60
   macro avg     0.6989    0.7647    0.7217        60
weighted avg     0.8650    0.8333    0.8451        60



In [13]:
mismatch = [i for i, (a,b) in enumerate(zip(Y_vir, vir_Y_test_binary)) if a != b]
print(mismatch)

[6, 14, 32, 34, 40, 43, 48, 49, 52, 58]


In [14]:
Y_hun = np.argmax(SGD_model.predict(hun_X_test),axis = 1)
cm_hun = confusion_matrix(Y_hun, hun_Y_test_binary)
print(cm_hun)
print(classification_report(Y_hun, hun_Y_test_binary,digits=4))

3/3 [==============================] - 0s 999us/step
[[53  5]
 [ 4 27]]
              precision    recall  f1-score   support

           0     0.9298    0.9138    0.9217        58
           1     0.8438    0.8710    0.8571        31

    accuracy                         0.8989        89
   macro avg     0.8868    0.8924    0.8894        89
weighted avg     0.8998    0.8989    0.8992        89



In [15]:
mismatch = [i for i, (a,b) in enumerate(zip(Y_hun, hun_Y_test_binary)) if a != b]
print(mismatch)

[12, 14, 22, 23, 29, 42, 46, 54, 73]


In [16]:
Y_swi = np.argmax(SGD_model.predict(swi_X_test),axis = 1)
cm_swi = confusion_matrix(Y_swi, swi_Y_test_binary)
print(cm_swi)
print(classification_report(Y_swi, swi_Y_test_binary, digits=4))

2/2 [==============================] - 0s 2ms/step
[[ 1  0]
 [ 1 35]]
              precision    recall  f1-score   support

           0     0.5000    1.0000    0.6667         1
           1     1.0000    0.9722    0.9859        36

    accuracy                         0.9730        37
   macro avg     0.7500    0.9861    0.8263        37
weighted avg     0.9865    0.9730    0.9773        37



In [17]:
mismatch = [i for i, (a,b) in enumerate(zip(Y_swi, swi_Y_test_binary)) if a != b]
print(mismatch)

[20]
